<a href="https://colab.research.google.com/github/AlexeyTri/PROJECT_SCOOL/blob/master/ActualGraphic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Условия:**
1. наличие AUTODESK DESKTOP
2. доступ ко всем таблицам в режиме РЕДАКТОР
3. все тактические графики должны быть приведены в гугл форму

# TODO

* парсинг таблицы ГРАФИК ЗАКАЗЧИКА из АСС
* парсинг договора подрядчика и автоматическая генерация ГУГЛ таблиц
* каждому виду работ в контрактном графике необходимо присвоить НАШ шифр
* график заказчика подгружается с локального компьютера и сохраняется туда же
* приветсти все данные к НОРМАЛЬНОМУ ТИПУ, включая индексы и названия колонок
* разобраться с процентами

# Библиотеки

In [1]:
import openpyxl
from openpyxl.cell.cell import WriteOnlyCell
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl import Workbook
import pandas as pd

In [7]:
idx = pd.IndexSlice

In [2]:
from google.colab import auth
auth.authenticate_user()
import re
import gspread
from google.auth import default
creds, _ = default()
import numpy as np
gc = gspread.authorize(creds)
from datetime import datetime

In [210]:
start_project = '01.02.2022'
finish_project = '02.09.2024'

# Обновление графика Закачика

* позиции имеющие шифр обновляются по шифру
* позиции без шифра обновляются после фильтрации по шифру

## предобработка графика Заказчика

In [ ]:
# файл загружается на ГУГЛ диск
from google.colab import files
upload = files.upload()

In [ ]:
# содежание текущей папки на ГУГЛ диске
!ls

ПАНДАС из редакции графика **заказчика**

In [5]:
df_graph = pd.read_excel(open('Сводный план-график 2022 12 28 иса.xlsx', 'rb'))

/usr/local/lib/python3.8/dist-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)


In [6]:
idx = pd.IndexSlice

In [ ]:
df_graph

In [ ]:
# stop
df_graph = pd.DataFrame(df_graph.values, columns=df_graph.values[3])
df_graph['Название задачи'] = df_graph['Название задачи'].map(lambda z: str(z).lstrip()).values
df_graph

In [ ]:
# stop
df_graph.drop([0,1,2,3], inplace=True)

In [160]:
#  список из графика заказчика
list2 = df_graph['Unnamed: 13']
list2 = list2.map(lambda z: str(z).lstrip())
list2 = list2.to_list()
list3 = df_graph['Unnamed: 2']
list3 = list3.map(lambda z: str(z).lstrip())
list3 = list3.to_list()

## обновление СМР

* пайплайн по каждому контракту СМР одинаковый

### предобработка графика работ подрядчика

In [208]:
worksheet_SMR = gc.open('План работ Договор Адамант')

In [209]:
graf = worksheet_SMR.worksheet("ГРАФИК РАБОТ")
graf = graf.get_all_values()
graf = np.array(graf)
graf =  pd.DataFrame(graf[1:, :])
graf

,0,1,2,3,4,5,6,7,8,9,...,282,283,284,285,286,287,288,289,290,291
0,PLAN,ГРАФИК РАБОТ,Начало работ по ГРП,Завершение работ по ГРП,Длительность,Ед.изм.,Объем по контракту,План на 13.01,Факт на 13.01,% выполнения плана,...,23.02.2023,23.02.2023,24.02.2023,24.02.2023,25.02.2023,25.02.2023,26.02.2023,26.02.2023,27.02.2023,27.02.2023
1,***,,,,,,,,,,...,план,факт,план,факт,план,факт,план,факт,план,факт
2,VZO_POS1_POD_MOB_.,Мобилизация,10.10.2022,10.05.2024,579,--,,,,"91,67%",...,,,,,,,,,,
3,VZO_POS1_POD_VRD_DP17.01,Временные дороги из дорожных плит (3х1.75),26.10.2022,18.02.2023,116,м2,300,"300,0","300,0","100,00%",...,,,,,,,,,,
4,VZO_POS1_POD_VRS_ES0.01,Временные электросети - установленные на начал...,23.11.2022,12.12.2022,20,комплект,2,"2,0","1,0","50,00%",...,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
397,BOT_GP___.,Работы по благоустройству,18.11.2022,05.04.2024,505,,,"0,0","0,0",,...,,,,,,,,,,
398,BOT_GP,Выдача рабочего проекта по разделу,18.11.2022,18.11.2022,1,РД,,,,,...,,,,,,,,,,
399,_EXP___.,Ввод объекта в эксплуатацию,14.03.2024,06.06.2024,85,,,"0,0","0,0",,...,,,,,,,,,,
400,_ZOS___.,ЗОС,14.03.2024,24.04.2024,42,,,"0,0","0,0",,...,,,,,,,,,,


### пересечение списков видов работ

In [ ]:
#  список из плана работ
list1 = graf.iloc[:, 0].values
list1

Пересечение списков задач плана работ и графика заказчика

In [ ]:
result = list(set(list1) & set(list2))
result

Обновляем данные в плане заказчика

In [ ]:
for i in result:
    df_graph.loc[df_graph['Unnamed: 13'].map(lambda z: str(z).lstrip()).values == i, 'Unnamed: 4'] = (pd.to_numeric(graf.loc[graf.iloc[:,0].map(lambda z: str(z).lstrip()).values == i, 9].values[0].replace(',', '.').replace('%', ''))/100)
    # проверка корректности внесения обновлений
    print({df_graph.loc[df_graph['Unnamed: 13'].map(lambda z: str(z).lstrip()).values == i, 'Unnamed: 2'].values[0]: pd.to_numeric(graf.loc[graf.iloc[:,0].map(lambda z: str(z).lstrip()).values == i, 9].values[0].replace(',', '.').replace('%', ''))/100})    

## обновление проектирования

* первый этап - обновить по шифрам
* второй этап - обновить по наименованию работ

In [89]:
worksheet_PD = gc.open('Реестр выдачи РД')

In [ ]:
graph_PD = worksheet_PD.worksheet('График ПИР ')
graph_PD = graph_PD.get_all_values()
graph_PD = np.array(graph_PD)
graph_PD = pd.DataFrame(graph_PD[7:])
graph_PD

In [ ]:
pd.Series(graph_PD.iloc[:,7]).unique()

In [91]:
perevod = {
    'STATUS A': 1,
    'STATUS B': 0.85,
    'STATUS C': 0.6,
    'WORK': 0.3,
    'START': 0.05,
    'HOLD': 0,
    '': ''
}

In [ ]:
list_PD_ID_1 = graph_PD[graph_PD.iloc[:, 1].values != ''].iloc[:,2]
list_PD_name = graph_PD[graph_PD.iloc[:, 1].values == ''].iloc[:,3]
list_PD = list_PD_ID_1.append(list_PD_name)
list_PD

In [169]:
result_PD_ID = list(set(list_PD) & set(list2))
result_PD_name = list(set(list_PD) & set(list3))

In [ ]:
for i in result_PD_ID:
    df_graph.loc[df_graph['Unnamed: 13'].map(lambda z: str(z).lstrip()).values == i, 'Unnamed: 4'] = perevod[graph_PD.loc[graph_PD.iloc[:,2] == i,7].values[0]]
    # проверка корректности внесения обновлений
    print({df_graph.loc[df_graph['Unnamed: 13'].map(lambda z: str(z).lstrip()).values == i, 'Unnamed: 2'].values[0]: perevod[graph_PD.loc[graph_PD.iloc[:,2] == i,7].values[0]]})

In [ ]:
for i in result_PD_name:
    df_graph.loc[df_graph['Unnamed: 2'].map(lambda z: str(z).lstrip()).values == i, 'Unnamed: 4'] = perevod[graph_PD.loc[graph_PD.iloc[:,3] == i,7].values[0]]
    print({df_graph.loc[df_graph['Unnamed: 2'].map(lambda z: str(z).lstrip()).values == i, 'Unnamed: 2'].values[0]: perevod[graph_PD.loc[graph_PD.iloc[:,3] == i,7].values[0]]}) 

In [222]:
df_graph.to_excel('Сводный план-график {} иса.xlsx'.format('11.01.2023'), sheet_name='Обновление от {}'.format('11.01.2023'),index=False)

# Обновление графика проектирования подрядчика

* выгрузка реестра замечаний 
* агрегирование данных по разделам
* заполнение графика проектирования

In [191]:
# файл загружается на ГУГЛ диск
from google.colab import files
upload = files.upload()

In [ ]:
value_essuse = pd.read_excel(open('Сводка по проблемам-202301121116.xlsx', 'rb'), sheet_name=1)
value_essuse.columns

In [ ]:
value_essuse = value_essuse[(value_essuse['Статус'] == 'Открыто') & (value_essuse['Тип'].isin(['Замечание к РД','Замечания к ПД']))]
value_essuse.loc[:, 'Размещение'] =  value_essuse['Размещение'].map(lambda x: str(x).split('.pdf')[0].split('_')[0])

In [72]:
value_essuse = value_essuse.loc[:, idx['Размещение','Тип замечания']]

In [ ]:
agg_func_count = {'Тип замечания': 'count'}
value_essuse = value_essuse.groupby(['Размещение','Тип замечания']).agg(agg_func_count)

In [ ]:
value_essuse

In [179]:
name_RD_issues = pd.Series(value_essuse.index.get_level_values(0)).unique()

In [180]:
graph_PD_RD = pd.DataFrame.from_records(worksheet_PD.worksheet('График ПИР ').get_all_values())

In [181]:
list_PD_RD = graph_PD_RD.iloc[:,2].to_list()

In [206]:
for i in name_RD_issues:
    try:
        number_row = list_PD_RD.index(i) + 1
        try:
            value_essuse.loc[i, 'Критическое замечание']
            worksheet_PD.worksheet('График ПИР ').update_cell(number_row, 8, 'STATUS C')
        except: 
            try:
                value_essuse.loc[i, 'Техническая ошибка'].values[0] != 0  
                value_essuse.loc[i, 'Недостаточно данных'].values[0] != 0
                worksheet_PD.worksheet('График ПИР ').update_cell(number_row, 8, 'STATUS B')
            except:
                try:
                    worksheet_PD.worksheet('График ПИР ').update_cell(number_row, 8, 'STATUS A') 
                except:
                    pass
    except:
        pass 

# Обновление графика СМР подрядчика

* в модели отмечается выполненный элемент
* данные попадают в GS + обновляется формы PLAN RADAR

## лист старта работ и кумулятивной суммы дня

In [212]:
new_graf = graf.copy()
idx = pd.IndexSlice
new_graf = new_graf.loc[idx[2:],idx[:6]]
new_graf.drop([4,5], axis=1, inplace=True)

In [213]:
new_graf.rename(columns = {0:'passwordCOST', 
                       1:'NAME',2:'START', 3:'FINISH', 6:'valueModel'}, 
            inplace = True)
new_graf = new_graf[new_graf['passwordCOST'] > '']
new_graf = new_graf[new_graf['START'] > '']
new_graf = new_graf[new_graf['FINISH'] > '']
new_graf = new_graf.set_index('passwordCOST')
new_graf['valueModel'] = new_graf['valueModel'].map(lambda x: pd.to_numeric(re.sub("[^0-9+,]", "", x).replace(',','.')))
new_graf = new_graf.fillna(0)

In [215]:
graf_name = graf[[0,1]][2:]
graf_name.rename(columns={0: 'passwordCOST', 1: 'NAME'}, inplace=True) 
graf_name = graf_name[graf_name['passwordCOST'] != 0]
graf_name = pd.DataFrame(graf_name['NAME'].values, index=graf_name['passwordCOST'], columns=['NAME'])

In [216]:
graf.drop(graf.index[1:10], axis=1, inplace=True)

In [217]:
newColumnsIndex = [pd.to_datetime(pd.Series(graf.iloc[0].values[1:]).apply(lambda x: str(x)), format='%d.%m.%Y'), graf.iloc[1][1:].values]

In [218]:
graf.rename(columns={0: 'passwordCOST'}, inplace=True)

In [219]:
graf = graf.set_index("passwordCOST")
graf = pd.DataFrame(graf.values, index=graf.index, columns=newColumnsIndex)
graf.drop(graf.index[:2], axis=0, inplace=True)
for i in graf.columns:
    graf[i] = graf[i].map(lambda x: pd.to_numeric(re.sub("[^0-9 +,]", "", x).replace(',','.')))

In [220]:
graf.fillna(0, inplace=True)
graf.sort_index(axis=0, level=1, ascending=True)
graf = graf[graf.index > '']
graf

2022-10-10      2022-10-11      2022-10-12       \
                               план факт       план факт       план факт   
passwordCOST                                                               
VZO_POS1_POD_MOB_.              0.0  0.0        0.0  0.0        0.0  0.0   
VZO_POS1_POD_VRD_DP17.01        0.0  0.0        0.0  0.0        0.0  0.0   
VZO_POS1_POD_VRS_ES0.01         0.0  0.0        0.0  0.0        0.0  0.0   
VZO_POS1_POD_VRS_VK0.01         0.0  0.0        0.0  0.0        0.0  0.0   
VZO_POS1_POD_VRD_DP17.02        0.0  0.0        0.0  0.0        0.0  0.0   
...                             ...  ...        ...  ...        ...  ...   
BOT_GP___.                      0.0  0.0        0.0  0.0        0.0  0.0   
BOT_GP                          0.0  0.0        0.0  0.0        0.0  0.0   
_EXP___.                        0.0  0.0        0.0  0.0        0.0  0.0   
_ZOS___.                        0.0  0.0        0.0  0.0        0.0  0.0   
_RNV___.                        0.0  0.0        0.0  0.0        0.0  0.0   

                         2022-10-13      2022-10-14       ... 2023-02-23       \
                               план факт       план факт  ...       план факт   
passwordCOST                                              ...                   
VZO_POS1_POD_MOB_.              0.0  0.0        0.0  0.0  ...        0.0  0.0   
VZO_POS1_POD_VRD_DP17.01        0.0  0.0        0.0  0.0  ...        0.0  0.0   
VZO_POS1_POD_VRS_ES0.01         0.0  0.0        0.0  0.0  ...        0.0  0.0   
VZO_POS1_POD_VRS_VK0.01         0.0  0.0        0.0  0.0  ...        0.0  0.0   
VZO_POS1_POD_VRD_DP17.02        0.0  0.0        0.0  0.0  ...        0.0  0.0   
...                             ...  ...        ...  ...  ...        ...  ...   
BOT_GP___.                      0.0  0.0        0.0  0.0  ...        0.0  0.0   
BOT_GP                          0.0  0.0        0.0  0.0  ...        0.0  0.0   
_EXP___.                        0.0  0.0        0.0  0.0  ...        0.0  0.0   
_ZOS___.                        0.0  0.0        0.0  0.0  ...        0.0  0.0   
_RNV___.                        0.0  0.0        0.0  0.0  ...        0.0  0.0   

                         2023-02-24      2023-02-25      2023-02-26       \
                               план факт       план факт       план факт   
passwordCOST                                                               
VZO_POS1_POD_MOB_.              0.0  0.0        0.0  0.0        0.0  0.0   
VZO_POS1_POD_VRD_DP17.01        0.0  0.0        0.0  0.0        0.0  0.0   
VZO_POS1_POD_VRS_ES0.01         0.0  0.0        0.0  0.0        0.0  0.0   
VZO_POS1_POD_VRS_VK0.01         0.0  0.0        0.0  0.0        0.0  0.0   
VZO_POS1_POD_VRD_DP17.02        0.0  0.0        0.0  0.0        0.0  0.0   
...                             ...  ...        ...  ...        ...  ...   
BOT_GP___.                      0.0  0.0        0.0  0.0        0.0  0.0   
BOT_GP                          0.0  0.0        0.0  0.0        0.0  0.0   
_EXP___.                        0.0  0.0        0.0  0.0        0.0  0.0   
_ZOS___.                        0.0  0.0        0.0  0.0        0.0  0.0   
_RNV___.                        0.0  0.0        0.0  0.0        0.0  0.0   

                         2023-02-27       
                               план факт  
passwordCOST                              
VZO_POS1_POD_MOB_.              0.0  0.0  
VZO_POS1_POD_VRD_DP17.01        0.0  0.0  
VZO_POS1_POD_VRS_ES0.01         0.0  0.0  
VZO_POS1_POD_VRS_VK0.01         0.0  0.0  
VZO_POS1_POD_VRD_DP17.02        0.0  0.0  
...                             ...  ...  
BOT_GP___.                      0.0  0.0  
BOT_GP                          0.0  0.0  
_EXP___.                        0.0  0.0  
_ZOS___.                        0.0  0.0  
_RNV___.                        0.0  0.0  

[383 rows x 282 columns]

In [225]:
start = start_project
start = pd.to_datetime(start)
finish = pd.to_datetime(pd.Timestamp.today().date())
start, finish

(Timestamp('2022-01-02 00:00:00'), Timestamp('2023-01-13 00:00:00'))

In [240]:
finish_yerstoday = finish - pd.Timedelta(1, "d")

In [242]:
sum_today = graf.loc[:, idx[finish:finish,'план']].sum(axis=1).values

In [244]:
total_plan = graf.loc[:, idx[start:finish,'план']].sum(axis=1)
cum_sum_yerstoday = graf.loc[:, idx[start:finish_yerstoday,'план']].sum(axis=1).values
sum_today = graf.loc[:, idx[finish:finish,'план']].sum(axis=1).values
# total_plan_value = graf.loc[:, idx[start:finish,'план']].sum(axis=1).values - graf.loc[:, idx[start:finish,'план']].sum(axis=1).values

In [238]:
total_plan = pd.DataFrame(total_plan,index=graf.index, columns=['cum_sum_today'])
total_plan
# кумулятивная сумма, план на сегодня, тригер

,cum_sum_today
passwordCOST,
VZO_POS1_POD_MOB_.,0.0
VZO_POS1_POD_VRD_DP17.01,300.0
VZO_POS1_POD_VRS_ES0.01,2.0
VZO_POS1_POD_VRS_VK0.01,1.0
VZO_POS1_POD_VRD_DP17.02,200.0
...,...
BOT_GP___.,0.0
BOT_GP,0.0
_EXP___.,0.0


## обновление плана недели